In [ ]:
import networkx as nx
# import matplotlib.pyplot as plt
import pygraphviz as pgv

from src.catalog_utils import load_catalog

In [ ]:
CATALOG = load_catalog()
CATALOG

In [ ]:
CATALOG.get_product("SW_MAGx_LR_1B").link_files_http

In [ ]:
products = list(CATALOG.products.keys())

In [ ]:
product_dependencies = {
    product: CATALOG.get_product(product).input_products
    for product in CATALOG.products.keys()
}
product_dependencies

In [ ]:
avail_cats ={
    "Advanced": "red",
    "Level1b": "black",
    "Level2daily": "green",
    "Level2longterm": "blue",
    "Multimission": "purple",
}
product_categories = {}
for product in CATALOG.products.keys():
    data_url = CATALOG.get_product(product).link_files_http
    for catname in avail_cats.keys():
        if catname in data_url:
            product_categories[product] = catname
            continue
# product_categories

In [ ]:
avail_fast = (
    # "SW_ACCx_PR_1B",
    "SW_ASMxAUX_1B",
    "SW_EFIx_LP_1B",
    "SW_EFIxLPI_1B",
    "SW_GPSxNAV_1B",
    "SW_GPSx_RN_1B",
    "SW_GPSx_RO_1B",
    "SW_LP_x_CA_1B",
    "SW_MAGx_CA_1B",
    "SW_MAGx_LR_1B",
    "SW_MAGx_HR_1B",
    "SW_MODx_SC_1B",
    "SW_SC_xDYN_1B",
    "SW_STRxATT_1B",
    "SW_VFMxAUX_1B",
)

In [ ]:
CATALOG.get_product("CH_DNS_ACC_2_").link_vires_gui

In [ ]:
def make_graph(product_dependencies=product_dependencies, product_categories=product_categories):

    # Build the graph
    G = pgv.AGraph(directed=True)

    # Add all nodes at once
    for node in product_dependencies.keys():
        cat = product_categories[node]
        color = avail_cats.get(cat, "grey")
        shape = "doubleoctagon" if node in avail_fast else "box"
        url = f"https://swarmhandbook.earth.esa.int/catalogue/{node}"
        link_notebook = CATALOG.get_product(node).link_notebook
        link_vires_gui = CATALOG.get_product(node).link_vires_gui
        bgcolor = "powderblue" if link_notebook or link_vires_gui else "white"
        G.add_node(
            node, shape=shape, color=color, URL=url, fillcolor=bgcolor,
            style="filled", fontsize=15, penwidth=3
        )

    # # Add each category of products as a cluster?
    # def _filter_nodes(cat):
    #     return [p for p, c in product_categories.items() if c==cat]
    # for cat, color in avail_cats.items():
    #     G.add_nodes_from(
    #         _filter_nodes(cat), color=color,# cluster=True
    #     )

    # Add dependency edges
    for node, neighbors in product_dependencies.items():
        for neighbor in neighbors:
            G.add_edge(node, neighbor)

        

    # Visualisation options
    # G.layout(prog="neato", args='-Gmaxiter=100')
    G.layout(prog="circo")

    return G

graph_viz = make_graph()
graph_viz.draw("network_graph.svg")
graph_viz


In [ ]:
def make_graph(product_dependencies=product_dependencies, product_categories=product_categories):

    # Build graph
    G = nx.DiGraph()
    for node in product_dependencies.keys():
        cat = product_categories[node]
        color = avail_cats.get(cat, "grey")
        G.add_nodes_from(
            [
                (node, {"color": color, "shape": "box", "fontsize": 5}),
            ]
        )
    for node, neighbors in product_dependencies.items():
        for neighbor in neighbors:
            G.add_edge(node, neighbor)

    pos = nx.spring_layout(G, seed=42)  # You can choose different layouts as well
    nx.draw(G, pos, with_labels=True, node_size=1500, node_color="skyblue", font_size=12, font_weight="bold")

    return G

make_graph()
